In [ ]:
!pip install playsound

In [ ]:
import cv2
import numpy as np
import smtplib
import playsound
import threading

In [ ]:
Alarm_Status = False
Email_Status = False
Fire_Reported = 0

In [ ]:
def play_alarm_sound_function():
    while True:
        playsound.playsound('Alarm Sound.mp3',True)

In [ ]:
def send_mail_function():

    recipientEmail = "kartik.sojitra96016@gmail.com"
    recipientEmail = recipientEmail.lower()

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login("kartik.sojitra5@gmail.com", 'Password')
        server.sendmail('kartik.sojitra5@gmail.com', "kartik.sojitra96016@gmail.com", "Warning A Fire Accident has been reported on XYZ Company")
        print("sent to {}".format(recipientEmail))
        server.close()
    except Exception as e:print(e)

In [ ]:
video = cv2.VideoCapture("Video.mp4") # If you want to use webcam use Index like 0,1.

In [ ]:
while True:
    (grabbed, frame) = video.read()
    if not grabbed:
        break

    frame = cv2.resize(frame, (960, 540)) 
 
    blur = cv2.GaussianBlur(frame, (21, 21), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
 
    lower = [18, 50, 50]
    upper = [35, 255, 255]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    mask = cv2.inRange(hsv, lower, upper)
 
    output = cv2.bitwise_and(frame, hsv, mask=mask)
    
    no_red = cv2.countNonZero(mask)
    if int(no_red) > 15000:
        Fire_Reported = Fire_Reported + 1
        cv2.imshow("output", output)
        if Fire_Reported >= 1:
            if Alarm_Status == False:
                threading.Thread(target=play_alarm_sound_function).start()
                Alarm_Status = True
                if Email_Status == False:
                    threading.Thread(target=send_mail_function).start()
                    Email_Status = True
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                          break

In [ ]:
cv2.destroyAllWindows()
video.release()